In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
from torch.autograd import Variable

sys.path.append('..')
sys.path.append('../visualization')
sys.path.append('../acd/util')
sys.path.append('../acd/scores')
sys.path.append('../acd/agglomeration')
import viz_1d as viz
import tiling_1d as tiling
import agg_1d as agg
import cd
import score_funcs


# form class to hold data
class B:
    def __init__(self):
        self.text = Variable(torch.zeros(1).cuda())

sys.path.append('../dsets/sst')
from dsets.sst import dset
sst_pkl = pickle.load(open('../dsets/sst/sst.pkl', 'rb'))

/accounts/grad/jmurdoch/.local/lib/python3.5/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


# load model

In [2]:
snapshot_file = '../dsets/sst/sst.model'
model = dset.get_model(snapshot_file).eval()

loading ../dsets/sst/sst.model


/usr/local/linux/anaconda3/lib/python3.5/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/linux/anaconda3/lib/python3.5/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/linux/anaconda3/lib/python3.5/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the objec

loaded onto gpu...


# run examples

In [3]:
# base parameters
sweep_dim = 1 # how large chunks of text should be considered (1 for words)
method = 'cd' # build_up, break_down, cd
percentile_include = 99.5 # keep this very high so we don't add too many words at once
num_iters = 25 # maximum number of iterations (rarely reached)

# text and label
sentence = ['a', 'great', 'ensemble', 'cast', 'ca', 'n\'t', 'lift', 'this', 'heartfelt', 'enterprise', 'out', 'of', 'the', 'familiar', '.'] # note this is a real example from the dataset
# sentence = ['not', 'good', ',', 'not', 'bad', ',', 'just', 'okay'] # any text with words from this dataset can be interpreted
label = 1 # 0 if positive 1 if negative


def batch_from_str_list(s):
    batch = B()
    nums = np.expand_dims(np.array([sst_pkl['stoi'][x] for x in s]).transpose(), axis=1)
    batch.text.data = torch.LongTensor(nums).cuda()
    return batch

# prepare inputs
batch = batch_from_str_list(sentence)
scores_all = model(batch).data.cpu().numpy()[0] # predict
label_pred = np.argmax(scores_all) # get predicted class

# agglomerate
lists = agg.agglomerate(model, batch, percentile_include, method, sweep_dim, # only works for sweep_dim = 1
                    sentence, label_pred, num_iters=num_iters) # see agg_1d.agglomerate to understand what this dictionary contains
lists = agg.collapse_tree(lists) # don't show redundant joins

# visualize
viz.word_heatmap(text_orig, lists, label_pred, label, fontsize=9)

../dsets/sst/model.py:29: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  lstm_out, self.hidden = self.lstm(vecs, self.hidden)
../acd/agglomeration/agg_1d.py:31: RuntimeWarning: invalid value encountered in greater_equal
  mask = scores >= thresh
/accounts/grad/jmurdoch/.local/lib/python3.5/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


NameError: name 'text_orig' is not defined